# Group PT to SOC 

Dased on MedDRA hierarchy.

PT (preferred terms)--> HLT (high-level term) --> HLGT (high-level group term) --> SOC (system organ class)


## Build MedDRA_dic in dataframe


In [2]:
from tqdm import tqdm

In [3]:
# read the MedDRA dics

import codecs
"""the key is code in string, the value is textual string"""
path_list = ['soc', 'hlgt', 'hlt', 'pt'] #
for i in range(len(path_list)):
    dic = {}
    path = '../Data/MedDRA/' + path_list[i] + '.asc'
    with codecs.open(path, encoding='utf-8-sig') as f:
        for line in f:
            s = line.split('$')
            if path_list[i] == 'soc':
                dic[s[0]] = [s[1].lower(), s[2]]
            else:
                dic[s[0]] = s[1].lower()
    locals()[path_list[i] +'_dic'] = dic


soc_dic['10017947'][1], soc_dic, len(soc_dic)

path_list = ['soc', 'hlgt', 'hlt', 'pt'] #, 'hlt', 'pt'
for i in range(len(path_list)-1):
    dic = {}
    path = '../Data/MedDRA/' + path_list[i] +'_' +path_list[i+1] + '.asc'
    print('workting on', path)
    with codecs.open(path, encoding='utf-8-sig') as f:
        for line in f:
            s = line.split('$')
            dic[s[1]] = s[0].lower()
    locals()[path_list[i+1]+'_' +path_list[i] +'_dic'] = dic
"""In the generated dic, the key is low level code(e.g., PT), the value is high-level code(e.g., hlt)"""

"""Build a big dataframe contains all PT, HLT, HLGT, SOC codes and their names"""
MedDRA_dic_all = pd.DataFrame()
column_name = ['PT', 'PT_name', 'HLT', 'HLT_name', 'HLGT', 'HLGT_name', 'SOC', 'SOC_name', 'SOC_abbr']
for key, value in tqdm(pt_dic.items()):
    hlt = pt_hlt_dic[key]  # find the corresponding hlt and hlt_name
    hlt_name = hlt_dic[hlt]
    hlgt = hlt_hlgt_dic[hlt]
    hlgt_name = hlgt_dic[hlgt]
    soc = hlgt_soc_dic[hlgt]
    soc_name = soc_dic[soc][0]
    soc_abbr = soc_dic[soc][1]
    lll = [key, value.lower(), hlt, hlt_name.lower(), hlgt, hlgt_name.lower(), soc, soc_name.lower(), soc_abbr]    
    
    
    lll_df = pd.DataFrame(np.array(lll).reshape(1,9),columns=column_name)
    MedDRA_dic_all = pd.concat([MedDRA_dic_all, lll_df])

pickle.dump(MedDRA_dic_all,  open('../Data/high_order/dic/MedDRA_dic_all.pk', 'wb'))

MedDRA_dic_all.shape, soc_name,hlgt_name, value, hlt_name

  0%|          | 0/24313 [00:00<?, ?it/s]

workting on ../Data/MedDRA/soc_hlgt.asc
workting on ../Data/MedDRA/hlgt_hlt.asc
workting on ../Data/MedDRA/hlt_pt.asc


100%|██████████| 24313/24313 [00:59<00:00, 409.98it/s]


((24313, 9),
 'investigations',
 'microbiology and serology investigations',
 'sars-cov-2 test false negative',
 'virus identification and serology')

### Build MedDRA_dic in dictionary


In [4]:
MedDRA_dic_all = pickle.load(open('../Data/high_order/dic/MedDRA_dic_all.pk', 'rb'))
MedDRA_dic_all.head()  

,PT,PT_name,HLT,HLT_name,HLGT,HLGT_name,SOC,SOC_name,SOC_abbr
0,10000002,11-beta-hydroxylase deficiency,10021608,inborn errors of steroid synthesis,10027424,metabolic and nutritional disorders congenital,10010331,"congenital, familial and genetic disorders",Cong
0,10000005,17 ketosteroids urine,10038589,reproductive hormone analyses,10014706,endocrine investigations (incl sex hormones),10022891,investigations,Inv
0,10000007,17 ketosteroids urine decreased,10038589,reproductive hormone analyses,10014706,endocrine investigations (incl sex hormones),10022891,investigations,Inv
0,10000009,17 ketosteroids urine increased,10038589,reproductive hormone analyses,10014706,endocrine investigations (incl sex hormones),10022891,investigations,Inv
0,10000011,17 ketosteroids urine normal,10038589,reproductive hormone analyses,10014706,endocrine investigations (incl sex hormones),10022891,investigations,Inv


In [5]:
MeDRA_dic = {}
PT_list = list(MedDRA_dic_all['PT_name'])
for i in tqdm(range(len(MedDRA_dic_all))):
    key = PT_list[i]
    value = list(MedDRA_dic_all.iloc[i])
    MeDRA_dic[key] = value

pickle.dump(MeDRA_dic, open('../Data/high_order/dic/MeDRA_dic.pk', 'wb'))
print('data saved with #-PT:', len(MeDRA_dic))

100%|██████████| 24313/24313 [00:03<00:00, 6774.97it/s]

data saved with #-PT: 24313


## Group our significant SE into SOC level

In [6]:
import seaborn as sns; sns.set()
MedDRA_dic_all = pickle.load(open('../Data/high_order/dic/MedDRA_dic_all.pk', 'rb'))

In [7]:
# MedDRA_dic_all[MedDRA_dic_all.PT_name == row.name].SOC
condition_list = ['SE_uncondition_2019_sig_over', 'SE_uncondition_2019_sig_under', 'SE_male_2019_sig_over', 'SE_male_2019_sig_under',
                 'SE_female_2019_sig_over', 'SE_female_2019_sig_under', 
                 'SE_young_2019_sig_over', 'SE_young_2019_sig_under', 'SE_adult_2019_sig_over', 'SE_adult_2019_sig_under',
                 'SE_elderly_2019_sig_over', 'SE_elderly_2019_sig_under']

for condition in condition_list:      
    print('condition', condition)
    locals()[condition] = pickle.load(open('../Data/pandemic/results/'+condition+'.pk', 'rb'))
    
    

    
    locals()[condition]['SOC'] = locals()[condition].apply(lambda row: MedDRA_dic_all[MedDRA_dic_all.PT_name == row['name']].SOC, axis = 1)
    locals()[condition]['SOC_name'] = locals()[condition].apply(lambda row: MedDRA_dic_all[MedDRA_dic_all.PT_name == row['name']].SOC_name, axis = 1)
    locals()[condition]['SOC_abbr'] = locals()[condition].apply(lambda row: MedDRA_dic_all[MedDRA_dic_all.PT_name == row['name']].SOC_abbr, axis = 1)

    #     Save the final dataframe that with SOC information
    pickle.dump(locals()[condition],  open('../Data/pandemic/results/'+condition+'_step2.pk', 'wb'))
    print(condition, 'saved')

condition SE_uncondition_2019_sig_over
SE_uncondition_2019_sig_over saved
condition SE_uncondition_2019_sig_under
SE_uncondition_2019_sig_under saved
condition SE_male_2019_sig_over
SE_male_2019_sig_over saved
condition SE_male_2019_sig_under
SE_male_2019_sig_under saved
condition SE_female_2019_sig_over
SE_female_2019_sig_over saved
condition SE_female_2019_sig_under
SE_female_2019_sig_under saved
condition SE_young_2019_sig_over
SE_young_2019_sig_over saved
condition SE_young_2019_sig_under
SE_young_2019_sig_under saved
condition SE_adult_2019_sig_over
SE_adult_2019_sig_over saved
condition SE_adult_2019_sig_under
SE_adult_2019_sig_under saved
condition SE_elderly_2019_sig_over
SE_elderly_2019_sig_over saved
condition SE_elderly_2019_sig_under
SE_elderly_2019_sig_under saved
